# CAPSTONE INVESTIGACIÓN OPERATIVA

In [9]:
#encender .venv
!.\.venv\Scripts\activate.bat

### Instalación de dependencias en el .venv

In [12]:
# Instala y actualiza los paquetes necesarios
%pip install --upgrade pip
%pip install numpy pandas scikit-learn matplotlib joblib gurobipy gurobi-machinelearning pandas-datareader xgboost

# Verifica la versión de scikit-learn y Python
import sklearn, sys
print("Python version:", sys.version)
print("scikit-learn version:", sklearn.__version__)

# Importa y verifica pandas_datareader
import importlib
import pandas_datareader as pdr
print("pandas_datareader OK:", importlib.metadata.version("pandas-datareader"))

Note: you may need to restart the kernel to use updated packages.
   ---------------------------------------- 0.0/56.8 MB ? eta -:--:--
    --------------------------------------- 0.8/56.8 MB 3.9 MB/s eta 0:00:15
   - -------------------------------------- 1.8/56.8 MB 4.8 MB/s eta 0:00:12
   -- ------------------------------------- 2.9/56.8 MB 4.6 MB/s eta 0:00:12
   -- ------------------------------------- 3.9/56.8 MB 4.8 MB/s eta 0:00:12
   --- ------------------------------------ 5.0/56.8 MB 4.8 MB/s eta 0:00:11
   ---- ----------------------------------- 6.3/56.8 MB 4.9 MB/s eta 0:00:11
   ----- ---------------------------------- 7.3/56.8 MB 5.0 MB/s eta 0:00:10
   ----- ---------------------------------- 8.4/56.8 MB 5.0 MB/s eta 0:00:10
   ------ --------------------------------- 9.7/56.8 MB 5.1 MB/s eta 0:00:10
   ------- -------------------------------- 10.7/56.8 MB 5.1 MB/s eta 0:00:10
   -------- ------------------------------- 11.8/56.8 MB 5.1 MB/s eta 0:00:09
   --------- --

### Importamos Librerias necesarias

In [13]:
# ============ Config & IO ============
import re
import time
import numpy as np
import pandas as pd

# ============ Scikit-learn ============
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
# ===== IMPORTS extra =====
from sklearn.base import clone           # por si no lo tenías
from xgboost import XGBRegressor         # pip install xgboost


# ============ Gurobi / MIP ============
import gurobipy as gp
from gurobipy import GRB
from gurobi_ml import add_predictor_constr

# ============ Datos externos (opcional) ============
from pandas_datareader.data import DataReader  # si usas FRED/CPI
from sklearn.base import clone

import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### Prepocesamiento de la base de datos "AmesHousing"

In [14]:
FILE = "AmesHousing(in).csv"

def clean_col(c: str) -> str:
    c = re.sub(r"[^0-9A-Za-z_]+", "_", c)
    c = re.sub(r"_+", "_", c).strip("_")
    return c

def rmse_compat(y_true, y_pred):
    try:
        return mean_squared_error(y_true, y_pred, squared=False)
    except TypeError:  # versiones viejas de sklearn
        return np.sqrt(mean_squared_error(y_true, y_pred))

# ====== 1) Cargar y ajustar por inflación ======
df = pd.read_csv(FILE)
df.columns = [clean_col(c) for c in df.columns]
assert {'Yr_Sold','Mo_Sold','SalePrice'}.issubset(df.columns)

sale_date = pd.to_datetime(dict(year=df['Yr_Sold'].astype(int),
                                month=df['Mo_Sold'].astype(int), day=1))
start = sale_date.min().replace(day=1)
today = pd.Timestamp.today().normalize()

try:
    cpi = DataReader('CPIAUCNS','fred', start, today).rename(columns={'CPIAUCNS':'CPI'})
    cpi.index = cpi.index.to_period('M').astype(str)
    TARGET_YM = max(cpi.index)
    CPI_TARGET = float(cpi.loc[TARGET_YM,'CPI'])
    df['_ym'] = sale_date.dt.to_period('M').astype(str)
    df['CPI_sale'] = df['_ym'].map(cpi['CPI'])
    y_real = df['SalePrice'].astype(float) * (CPI_TARGET / df['CPI_sale'])
    print(f"[OK] SalePrice ajustado a base {TARGET_YM}.")
except Exception as e:
    print("[AVISO] No se pudo consultar CPI FRED; usaré SalePrice sin ajustar. Detalle:", e)
    TARGET_YM = f"{int(df['Yr_Sold'].max())}-{int(df['Mo_Sold'].max()):02d}"
    y_real = df['SalePrice'].astype(float)

y_log = np.log1p(y_real)

# ====== 2) Features MIP-friendly (numéricas + ordinales; sin OHE) ======
qual_map = {'Po':1,'Fa':2,'TA':3,'Gd':4,'Ex':5}

kq_col = 'Kitchen_Qual' if 'Kitchen_Qual' in df.columns else ('KitchenQual' if 'KitchenQual' in df.columns else None)
if kq_col: df['KitchenQual_ord'] = df[kq_col].map(qual_map)

if 'Garage_Finish' in df.columns:
    df['GarageFinish_ord'] = df['Garage_Finish'].map({'Unf':1,'RFn':2,'Fin':3})

feats = [c for c in [
    'Gr_Liv_Area','Total_Bsmt_SF','Garage_Area','Garage_Cars',
    'Full_Bath','Fireplaces','Overall_Qual','KitchenQual_ord','GarageFinish_ord', 'Mo_Sold'
] if c in df.columns]


X = df[feats].copy()  
medians = X.median(numeric_only=True)
X   = X.fillna(medians).copy()     # <- sin NaN
# ====== 3) Modelos (Pipelines): Lineal y GBR ======
# ====== 3) Modelos (Pipelines): Lineal, GBR y XGBoost ======
models = {
    "Linear": Pipeline([
        ("scaler", StandardScaler()),
        ("est", LinearRegression())
    ]),
    "GBR": Pipeline([
        ("scaler", StandardScaler()),
        ("est", GradientBoostingRegressor(
            learning_rate=0.05, n_estimators=400, max_depth=3,
            subsample=0.9, random_state=42
        ))
    ]),
    "XGB": Pipeline([
        ("scaler", "passthrough"),   # árboles no necesitan escalado
        ("est", XGBRegressor(
            objective="reg:squarederror",
            n_estimators=600,
            learning_rate=0.05,
            max_depth=4,
            subsample=0.8,
            colsample_bytree=0.8,
            min_child_weight=2,
            reg_lambda=5.0,
            n_jobs=-1,
            tree_method="hist",
            random_state=42
        ))
    ])
}


def cv_eval(estimator, X, y_log, folds=5, seed=42):
    kf = KFold(n_splits=folds, shuffle=True, random_state=seed)
    y_true, y_pred = [], []
    for tr, te in kf.split(X):
        Xt, Xv = X.iloc[tr], X.iloc[te]
        yt, yv = y_log.iloc[tr], y_log.iloc[te]
        m = clone(estimator)
        m.fit(Xt, yt)
        pv = np.expm1(m.predict(Xv))  # de log a precio real
        y_true.append(np.expm1(yv)); y_pred.append(pv)
    y_true = np.concatenate(y_true); y_pred = np.concatenate(y_pred)
    return {
        "RMSE": rmse_compat(y_true, y_pred),
        "MAE": mean_absolute_error(y_true, y_pred),
        "R2": r2_score(y_true, y_pred),
    }

# ====== 4) Evaluación y entrenamiento final ======
rows, fitted = [], {}
for name, est in models.items():
    metrics = cv_eval(est, X, y_log)
    rows.append({"model": name, **{k: round(v,3) for k,v in metrics.items()}})
    est.fit(X, y_log)          # entrena full para el “modelo final”
    fitted[name] = est

results = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)
print(results)

best_name = results.iloc[0]["model"]
best_model = fitted[best_name]
print(f"\nGanador por RMSE (CV): {best_name}")

# ====== 5) Interpretabilidad básica para el informe ======
# ====== 5) Interpretabilidad básica para el informe ======
if best_name == "Linear":
    lr = best_model.named_steps["est"]
    coefs = pd.Series(lr.coef_.ravel(), index=feats).sort_values(key=np.abs, ascending=False)
    print("\nCoeficientes (ordenados por |coef|) — escala estandarizada:")
    print(coefs.to_string())
else:
    # Sirve tanto para GBR como para XGB
    tree_est = best_model.named_steps["est"]
    imps = pd.Series(tree_est.feature_importances_, index=feats).sort_values(ascending=False)
    print(f"\nImportancias de características ({best_name}):")
    print(imps.to_string())


# Para usar más tarde en Gurobi:
trained_feats = feats
trained_stats = pd.DataFrame({"q05": X.quantile(0.05), "median": X.median(), "q95": X.quantile(0.95)})
best_linear_model = fitted["Linear"]
best_gbr_model    = fitted["GBR"]
TARGET_YM = TARGET_YM  # mes base del ajuste (si se logró)


[OK] SalePrice ajustado a base 2025-07.
    model       RMSE        MAE     R2
0     GBR  43359.656  28906.987  0.876
1     XGB  44141.342  29314.731  0.871
2  Linear  70278.949  32186.723  0.674

Ganador por RMSE (CV): GBR

Importancias de características (GBR):
Overall_Qual        0.520330
Gr_Liv_Area         0.166957
Total_Bsmt_SF       0.096788
KitchenQual_ord     0.084411
Garage_Cars         0.047412
Garage_Area         0.040403
Fireplaces          0.023040
GarageFinish_ord    0.010904
Full_Bath           0.005960
Mo_Sold             0.003795


In [15]:


for (index, fila), y in zip(df.iterrows(), y_real):
    print(y, fila.SalePrice, fila.SalePrice * (CPI_TARGET / fila.CPI_sale), fila.Yr_Sold, fila.Mo_Sold, fila.CPI_sale)


318342.45432628406 215000 318342.45432628406 2010 5 218.178
155621.4988644966 105000 155621.4988644966 2010 6 217.965
254922.83623517535 172000 254922.83623517535 2010 6 217.965
361561.73368989356 244000 361561.73368989356 2010 4 218.009
281884.5440217616 189900 281884.5440217616 2010 3 217.631
289752.4097905627 195500 289752.4097905627 2010 6 217.965
316366.5169786569 213500 316366.5169786569 2010 4 218.009
285497.9394241463 191500 285497.9394241463 2010 1 216.687
351056.84392388957 236500 351056.84392388957 2010 3 217.631
280118.6979560938 189000 280118.6979560938 2010 6 217.965
260650.44654119786 175900 260650.44654119786 2010 4 218.009
274611.0618432117 185000 274611.0618432117 2010 3 217.631
267111.52911842626 180400 267111.52911842626 2010 5 218.178
255617.22055356394 171500 255617.22055356394 2010 2 216.741
314207.2167549836 212000 314207.2167549836 2010 6 217.965
797374.9179914206 538000 797374.9179914206 2010 6 217.965
243065.9601312137 164000 243065.9601312137 2010 6 217.965


### Parámetros del modelo

In [ ]:
# ----------------------------
# 2) Baseline, límites y costos (FIX)
# ----------------------------
# Requiere que ya existan:
# X, y_real, trained_feats, trained_stats (q05/median/q95), best_linear_model, best_gbr_model

ID_CASA = 987  # índice por fila (iloc)

# 1) Baseline: misma estructura (columnas) que se usó al entrenar
n = len(X)
idx = ID_CASA if 0 <= ID_CASA < n else 0  # fallback si se pasa de rango
baseline = X.iloc[idx].astype(float)

print("Caso base (features):")
print(baseline.to_string())

# 2) Elige el predictor (pipeline) que quieras evaluar como baseline
#    Puedes cambiar a best_linear_model si prefieres la lineal
PREDICTOR = best_gbr_model

# Predicción en log (nuestros modelos entrenan sobre y_log)
pred_log = float(PREDICTOR.predict(baseline.to_frame().T)[0])
precio_base_pred = float(np.expm1(pred_log))
print(f"\nPrecio base predicho: {precio_base_pred:,.0f}")

# Precio real de esa fila (en escala original; ya inflacionado si ajustaste antes)
precio_real = float(y_real.iloc[idx])
print(f"Precio real observado: {precio_real:,.0f}")

# 3) Cuantil 95% por feature (para cotas superiores)
q95 = trained_stats["q95"]  # viene de: trained_stats = DataFrame({q05, median, q95})

# 4) Costos por unidad (revisa unidades: p. ej., Gr_Liv_Area está en ft²)
default_costs = {
    "Gr_Liv_Area":     200,   # USD por ft²
    "Total_Bsmt_SF":    80,   # USD por ft²
    "Garage_Area":      60,   # USD por ft²
    "Garage_Cars":   17000,   # USD por puesto
    "Full_Bath":     25000,   # USD por baño
    "Fireplaces":      6000,  # USD por chimenea
    "Overall_Qual":   20000,  # USD por subir 1 punto (proxy)
    "KitchenQual_ord": 8000,  # USD por subir 1 nivel (TA→Gd→Ex) (ejemplo)
    "GarageFinish_ord":4000,  # USD por subir 1 nivel (Unf→RFn→Fin) (ejemplo)
}

# 5) "Room to grow" máximo (cuánto puedes mover vs baseline, si la norma/lote lo permite)
room = {
    "Gr_Liv_Area":     400.0,
    "Total_Bsmt_SF":   300.0,
    "Garage_Area":     250.0,
    "Garage_Cars":       1.0,   # sube a 2 si el lote lo permite
    "Full_Bath":         1.0,
    "Fireplaces":        1.0,
    "Overall_Qual":      1.0,
    "KitchenQual_ord":   1.0,
    "GarageFinish_ord":  1.0,
}

# 6) Construir cotas y costos por feature (solo para las usadas al entrenar)
bounds, costs = {}, {}
for f in trained_feats:
    base = float(baseline.get(f, X[f].median()))
    ub_by_room = base + room.get(f, 0.0)          # límite por “capacidad de cambio”
    ub_by_p95  = float(q95.get(f, base))          # límite por mercado (evita extrapolación)
    lb = base                                     # no bajamos de lo que ya tiene la casa base
    ub = max(lb, min(ub_by_room, ub_by_p95))      # asegura lb <= ub
    bounds[f] = (lb, ub)
    costs[f]  = float(default_costs.get(f, 0.0))

BUDGET = 200_000  # mismo orden de magnitud que tus costos (USD en este ejemplo)

print("\nLímites por feature (lb, ub):")
for f, (lb, ub) in bounds.items():
    if ub > lb:
        print(f"  {f:>16}: {lb:,.2f} → {ub:,.2f}")
    else:
        print(f"  {f:>16}: {lb:,.2f} (sin margen de aumento por p95/room)")

print("\nCostos unitarios usados:")
for f in trained_feats:
    if costs[f] > 0:
        print(f"  {f:>16}: {costs[f]:,.0f}")


Caso base (features):
Gr_Liv_Area         1567.0
Total_Bsmt_SF       1567.0
Garage_Area          714.0
Garage_Cars            2.0
Full_Bath              2.0
Fireplaces             2.0
Overall_Qual           6.0
KitchenQual_ord        4.0
GarageFinish_ord       2.0
Mo_Sold                5.0

Precio base predicho: 339,402
Precio real observado: 296,075

Límites por feature (lb, ub):
       Gr_Liv_Area: 1,567.00 → 1,967.00
     Total_Bsmt_SF: 1,567.00 → 1,776.00
       Garage_Area: 714.00 → 856.00
       Garage_Cars: 2.00 → 3.00
         Full_Bath: 2.00 (sin margen de aumento por p95/room)
        Fireplaces: 2.00 (sin margen de aumento por p95/room)
      Overall_Qual: 6.00 → 7.00
   KitchenQual_ord: 4.00 → 5.00
  GarageFinish_ord: 2.00 → 3.00
           Mo_Sold: 5.00 (sin margen de aumento por p95/room)

Costos unitarios usados:
       Gr_Liv_Area: 200
     Total_Bsmt_SF: 80
       Garage_Area: 60
       Garage_Cars: 17,000
         Full_Bath: 25,000
        Fireplaces: 6,000
      Ove

### Guroby

In [ ]:
# ---------- Parámetros ----------
MODEL_NAME     = "auto"          # "auto" | "Linear" | "GBR"
OBJECTIVE_MODE = "profit"        # "price" | "profit" | "roi"
ROI_MIN        = 0.10            # usado sólo si OBJECTIVE_MODE == "roi"
PWL_K          = 25              # segmentos para PWL (log1p -> price)

# ---------- Resolver el estimador (con nuestro flujo Linear/GBR) ----------
# Requiere: best_linear_model, best_gbr_model, (opcional) results con RMSE
available = {}
if 'best_linear_model' in globals(): available["Linear"] = best_linear_model
if 'best_gbr_model'    in globals(): available["GBR"]    = best_gbr_model
assert available, "No encuentro modelos entrenados (best_linear_model / best_gbr_model)."

if MODEL_NAME == "auto":
    if 'results' in globals():
        # elige el mejor entre los disponibles por RMSE
        ok = results[results["model"].isin(available.keys())].sort_values("RMSE")
        chosen = ok.iloc[0]["model"] if len(ok) else next(iter(available))
    else:
        # fallback: prioriza GBR si existe
        chosen = "GBR" if "GBR" in available else "Linear"
else:
    assert MODEL_NAME in available, f"'{MODEL_NAME}' no está disponible. Usa 'Linear' o 'GBR'."
    chosen = MODEL_NAME

predictor = available[chosen]



print(f"→ Usando modelo: {chosen}")

# ---------- Columnas esperadas ----------
assert 'trained_feats' in globals(), "Falta 'trained_feats' (columnas de entrenamiento)."
feat_names_model = list(trained_feats)

for need in ("baseline","bounds","costs","BUDGET"):
    assert need in globals(), f"Falta '{need}' en el entorno."

missing = [c for c in feat_names_model if c not in bounds or c not in costs or c not in baseline.index]
assert not missing, f"Faltan claves en bounds/baseline/costs para: {missing}"

# ---------- Modelo de optimización ----------
m = gp.Model(f"ames_renov_opt_{chosen}")
int_like = {"Garage_Cars","Full_Bath","Fireplaces","Overall_Qual","KitchenQual_ord","GarageFinish_ord"}
x = {}
for c in feat_names_model:
    lb, ub = bounds[c]
    if c in int_like:
        lb_i, ub_i = int(np.floor(lb)), int(np.ceil(ub))
        if ub_i < lb_i: ub_i = lb_i
        x[c] = m.addVar(lb=lb_i, ub=ub_i, vtype=GRB.INTEGER,   name=c)
    else:
        x[c] = m.addVar(lb=float(lb), ub=float(ub), vtype=GRB.CONTINUOUS, name=c)

# Presupuesto (sólo mejoras; ya aseguramos lb = baseline en bounds)
cost_expr = gp.quicksum(costs[c] * (x[c] - float(baseline[c])) for c in feat_names_model)
m.addConstr(cost_expr <= float(BUDGET), name="Budget")

# (Opcional) ejemplo garaje acoplado por área
# STALL_SF = 200.0
# if "Garage_Area" in x and "Garage_Cars" in x:
#     m.addConstr((x["Garage_Area"] - float(baseline["Garage_Area"])) >=
#                 STALL_SF * (x["Garage_Cars"] - float(baseline["Garage_Cars"])),
#                 name="GarageAreaCoversStalls")

# DF 1xN con el orden exacto de columnas
x_df = pd.DataFrame([[x[c] for c in feat_names_model]], columns=feat_names_model)

# Variable de salida del predictor (log1p del precio)
y_pred_log = m.addVar(name="y_pred_log")

add_predictor_constr(
    gp_model=m,
    predictor=predictor,  # pipeline Linear o GBR
    input_vars=x_df,
    output_vars=y_pred_log
)

# ---------- Objetivo ----------
if OBJECTIVE_MODE == "price":
    # Maximiza log1p(price): suficiente porque exp() es monótona
    m.setObjective(y_pred_log, GRB.MAXIMIZE)

else:
    # Precio en escala original: price ≈ exp(y_log) - 1 vía PWL
    assert 'y_log' in globals(), "Falta 'y_log' para acotar PWL."
    ymin = float(np.percentile(y_log, 1))
    ymax = float(np.percentile(y_log, 99))
    if not np.isfinite(ymin) or not np.isfinite(ymax) or ymin >= ymax:
        ymin, ymax = 10.5, 13.5  # fallback razonable

    xs = np.linspace(ymin, ymax, int(max(2, PWL_K)) ).tolist()
    ys = (np.expm1(xs)).tolist()

    price = m.addVar(name="price")  # misma moneda que y_real
    m.addGenConstrPWL(y_pred_log, price, xs, ys, name="log_to_price")

    # Precio baseline (constante, mismo predictor)
    baseline_vec = pd.DataFrame([baseline[feat_names_model].to_dict()], columns=feat_names_model)
    price_before = float(np.expm1(predictor.predict(baseline_vec))[0])

    if OBJECTIVE_MODE == "profit":
        # Max (precio_after - costo)
        m.setObjective(price - cost_expr, GRB.MAXIMIZE)
    elif OBJECTIVE_MODE == "roi":
        # Max (precio_after - costo) con ROI mínimo
        m.addConstr(price - price_before >= ROI_MIN * cost_expr, name="ROImin")
        m.setObjective(price - cost_expr, GRB.MAXIMIZE)
    else:
        raise ValueError("OBJECTIVE_MODE debe ser 'price', 'profit' o 'roi'.")

# ---------- Resolver ----------
m.Params.OutputFlag = 1
m.optimize()

# ---------- Reporte ----------
if m.SolCount > 0:
    baseline_vec = pd.DataFrame([baseline[feat_names_model].to_dict()], columns=feat_names_model)
    price_before_rep = float(np.expm1(predictor.predict(baseline_vec))[0])

    if OBJECTIVE_MODE == "price":
        price_after_rep = float(np.expm1(y_pred_log.X))  # aprox (sin PWL)
    else:
        price_after_rep = float(price.X)                 # PWL exacta

    deltas = {c: x[c].X - float(baseline[c]) for c in feat_names_model}
    spent  = float(cost_expr.getValue())
    profit = price_after_rep - price_before_rep - spent
    roi    = (profit / spent) if spent > 0 else float('nan')

    print("\n=== SOLUCIÓN ===")
    print(f"Modelo usado    : {chosen}")
    print(f"Objetivo        : {OBJECTIVE_MODE}")
    print(f"Precio antes    : {price_before_rep:,.0f}")
    print(f"Precio después  : {price_after_rep:,.0f}")
    print(f"Gasto estimado  : {spent:,.0f}  (<= presupuesto {BUDGET:,.0f})")
    print(f"Ganancia        : {profit:,.0f}   |  ROI: {roi:,.2f}")

    print("\nValores finales de features:")
    for c in feat_names_model:
        print(f" - {c:15s}: {x[c].X:10.3f}  (base {float(baseline[c]):10.3f}, Δ={deltas[c]:+10.3f})")
else:
    print("No se encontró solución factible.")


→ Usando modelo: GBR
Set parameter Username
Set parameter LicenseID to value 2670211
Academic license - for non-commercial use only - expires 2026-05-23
Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 1212 rows, 3543 columns and 4351 nonzeros
Model fingerprint: 0x5c96ec7f
Model has 11550 simple general constraints
  11549 INDICATOR, 1 PWL
Variable types: 416 continuous, 3127 integer (3121 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+04]
  Objective range  [1e+00, 3e+04]
  Bounds range     [1e+00, 2e+03]
  RHS range        [3e-03, 9e+05]
  PWLCon x range   [1e+01, 1e+01]
  PWLCon y range   [1e+05, 6e+05]
  GenCon rhs range [7e-07, 1e+01]
  GenCon coe range [1e+00, 1e+00]
Presolve added 2675 rows and 0 colum